In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
image_size = (224, 224)
batch_size = 32


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [4]:
train_generator = train_datagen.flow_from_directory(
    '/Users/nivedha/Downloads/archive-2/real_vs_fake/real-vs-fake/train',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'  
)

Found 100000 images belonging to 2 classes.


In [5]:
validation_datagen = ImageDataGenerator(rescale=1.0/255)


In [6]:
validation_generator = validation_datagen.flow_from_directory(
    '/Users/nivedha/Downloads/archive-2/real_vs_fake/real-vs-fake/valid',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [7]:
base_model = ResNet50(weights='imagenet', include_top=False)


Metal device set to: Apple M1


2023-09-25 00:37:13.724422: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-25 00:37:13.724761: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [9]:
model = Model(inputs=base_model.input, outputs=predictions)


In [10]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [11]:
epochs = 10


In [12]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

Epoch 1/10


2023-09-25 00:37:15.562656: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-25 00:37:17.175550: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3125/3125 [==============================] - ETA: 0s - loss: 0.4516 - accuracy: 0.7842

2023-09-25 01:56:48.846453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3125/3125 [==============================] - 5070s 2s/step - loss: 0.4516 - accuracy: 0.7842 - val_loss: 0.6796 - val_accuracy: 0.7158
Epoch 2/10
3125/3125 [==============================] - 4938s 2s/step - loss: 0.2466 - accuracy: 0.8980 - val_loss: 1.7046 - val_accuracy: 0.6093
Epoch 3/10
3125/3125 [==============================] - 8240s 3s/step - loss: 0.1799 - accuracy: 0.9280 - val_loss: 0.3359 - val_accuracy: 0.8553
Epoch 4/10
3125/3125 [==============================] - 6617s 2s/step - loss: 0.1486 - accuracy: 0.9414 - val_loss: 0.1432 - val_accuracy: 0.9445
Epoch 5/10
3125/3125 [==============================] - 5034s 2s/step - loss: 0.1252 - accuracy: 0.9516 - val_loss: 0.2416 - val_accuracy: 0.9090
Epoch 6/10
3125/3125 [==============================] - 6143s 2s/step - loss: 0.1047 - accuracy: 0.9595 - val_loss: 0.4248 - val_accuracy: 0.8071
Epoch 7/10
3125/3125 [==============================] - 6381s 2s/step - loss: 0.0926 - accuracy: 0.9647 - val_loss: 1.4504 - val_accura

In [14]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [15]:
test_generator = test_datagen.flow_from_directory(
    '/Users/nivedha/Downloads/archive-2/real_vs_fake/real-vs-fake/test',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 20000 images belonging to 2 classes.


In [16]:
predictions = model.predict(test_generator)
predicted_classes = np.round(predictions)


2023-09-25 18:16:34.558387: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 215s 342ms/step


In [17]:
true_classes = test_generator.classes

In [18]:
print("Confusion Matrix:")
print(confusion_matrix(true_classes, predicted_classes))


Confusion Matrix:
[[10000     0]
 [10000     0]]


In [19]:
print("\nClassification Report:")
print(classification_report(true_classes, predicted_classes, target_names=train_generator.class_indices))


Classification Report:
              precision    recall  f1-score   support

        fake       0.50      1.00      0.67     10000
        real       0.00      0.00      0.00     10000

    accuracy                           0.50     20000
   macro avg       0.25      0.50      0.33     20000
weighted avg       0.25      0.50      0.33     20000



/Users/nivedha/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nivedha/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nivedha/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera